In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv('./train.csv')

In [3]:
data

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,NaN,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10.0,Australia Other,South Eastern Australia,NaN,Andrew Peace,Chardonnay
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,NaN,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest,Red Blend
2,NaN,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80.0,Piedmont,Barolo,NaN,Renato Ratti,Nebbiolo
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22.0,Southwest France,Bergerac Sec,NaN,Domaine l'Ancienne Cure,Bordeaux-style White Blend
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33.0,France Other,Vin de Liqueur,NaN,Château du Cèdre,Malbec
...,...,...,...,...,...,...,...,...,...,...,...,...
82652,@wineschach,Spain,Montecillo 2007 Crianza (Rioja),"Hot earth, baked cherry and mild vanilla aroma...",Crianza,84,12.0,Northern Spain,Rioja,NaN,Montecillo,Tempranillo
82653,NaN,US,Vivác Winery 2008 Single Vineyard Cabernet Sau...,"Pungent, high-pitched notes of turmeric and dr...",Single Vineyard,85,21.0,New Mexico,New Mexico,NaN,Vivác Winery,Cabernet Sauvignon
82654,@vossroger,France,Domaine de la Madone 2014 Nouveau (Beaujolais...,This is a wine that's not just lively and frui...,Nouveau,87,14.0,Beaujolais,Beaujolais-Villages,NaN,Domaine de la Madone,Gamay
82655,@vossroger,France,Château Beaumont 2011 Haut-Médoc,"An impressive wine, it already shows signs of ...",NaN,91,20.0,Bordeaux,Haut-Médoc,NaN,Château Beaumont,Bordeaux-style Red Blend


In [0]:
from sklearn_pandas import CategoricalImputer
imputer = CategoricalImputer(missing_values = 'NaN', strategy = 'most_frequent')
for i in range(12):
    if i in [2,3]:
        continue
    imputer = imputer.fit(data.iloc[:,i])
    data.iloc[:,i] = imputer.transform(data.iloc[:,i])

In [5]:
data

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,@vossroger,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10.0,Australia Other,South Eastern Australia,Central Coast,Andrew Peace,Chardonnay
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,Reserve,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest,Red Blend
2,@vossroger,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80.0,Piedmont,Barolo,Central Coast,Renato Ratti,Nebbiolo
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22.0,Southwest France,Bergerac Sec,Central Coast,Domaine l'Ancienne Cure,Bordeaux-style White Blend
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33.0,France Other,Vin de Liqueur,Central Coast,Château du Cèdre,Malbec
...,...,...,...,...,...,...,...,...,...,...,...,...
82652,@wineschach,Spain,Montecillo 2007 Crianza (Rioja),"Hot earth, baked cherry and mild vanilla aroma...",Crianza,84,12.0,Northern Spain,Rioja,Central Coast,Montecillo,Tempranillo
82653,@vossroger,US,Vivác Winery 2008 Single Vineyard Cabernet Sau...,"Pungent, high-pitched notes of turmeric and dr...",Single Vineyard,85,21.0,New Mexico,New Mexico,Central Coast,Vivác Winery,Cabernet Sauvignon
82654,@vossroger,France,Domaine de la Madone 2014 Nouveau (Beaujolais...,This is a wine that's not just lively and frui...,Nouveau,87,14.0,Beaujolais,Beaujolais-Villages,Central Coast,Domaine de la Madone,Gamay
82655,@vossroger,France,Château Beaumont 2011 Haut-Médoc,"An impressive wine, it already shows signs of ...",Reserve,91,20.0,Bordeaux,Haut-Médoc,Central Coast,Château Beaumont,Bordeaux-style Red Blend


In [0]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [0]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(y)

In [8]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10
0,@vossroger,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10,Australia Other,South Eastern Australia,Central Coast,Andrew Peace
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,Reserve,89,15,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest
2,@vossroger,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80,Piedmont,Barolo,Central Coast,Renato Ratti
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22,Southwest France,Bergerac Sec,Central Coast,Domaine l'Ancienne Cure
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33,France Other,Vin de Liqueur,Central Coast,Château du Cèdre
...,...,...,...,...,...,...,...,...,...,...,...
82652,@wineschach,Spain,Montecillo 2007 Crianza (Rioja),"Hot earth, baked cherry and mild vanilla aroma...",Crianza,84,12,Northern Spain,Rioja,Central Coast,Montecillo
82653,@vossroger,US,Vivác Winery 2008 Single Vineyard Cabernet Sau...,"Pungent, high-pitched notes of turmeric and dr...",Single Vineyard,85,21,New Mexico,New Mexico,Central Coast,Vivác Winery
82654,@vossroger,France,Domaine de la Madone 2014 Nouveau (Beaujolais...,This is a wine that's not just lively and frui...,Nouveau,87,14,Beaujolais,Beaujolais-Villages,Central Coast,Domaine de la Madone
82655,@vossroger,France,Château Beaumont 2011 Haut-Médoc,"An impressive wine, it already shows signs of ...",Reserve,91,20,Bordeaux,Haut-Médoc,Central Coast,Château Beaumont


In [0]:
X = X[:,2]+X[:,3]

In [10]:
pd.DataFrame(X)

,0
0,Andrew Peace 2007 Peace Family Vineyard Chardo...
1,North by Northwest 2014 Red (Columbia Valley (...
2,Renato Ratti 2007 Conca (Barolo)Barolo Conca ...
3,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...
4,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...
...,...
82652,"Montecillo 2007 Crianza (Rioja)Hot earth, bak..."
82653,Vivác Winery 2008 Single Vineyard Cabernet Sau...
82654,Domaine de la Madone 2014 Nouveau (Beaujolais...
82655,Château Beaumont 2011 Haut-MédocAn impressive...


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [88]:
from sklearn.feature_extraction.text import CountVectorizer 
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(X) 
X_counts.shape

(82657, 46242)

In [89]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=False)
X_tfidf = tfidf_transformer.fit_transform(X_counts)
X_tfidf.shape

(82657, 46242)

In [90]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
clf_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf", LinearSVC())])
clf_pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                     

In [91]:
#Train Accuracy
y_pred = clf_pipe.predict(X)
np.mean(y_pred == y)

0.9961648741183445

In [86]:
#Test Accuracy
y_pred = clf_pipe.predict(X_test)
np.mean(y_pred == y_test)

0.960379869344302

In [0]:
data1 = pd.read_csv('./test.csv')

In [0]:
test = data1.iloc[:,:].values
test = test[:,2] + " " + test[:,3]

In [96]:
test

array(['Boedecker Cellars 2011 Athena Pinot Noir (Willamette Valley) Nicely differentiated from the companion Stewart bottling, this Athena is a sharp and spicy affair, with tight, tart cranberry and raspberry fruit. Native yeast fermentation and extended sur-lie aging add some snap to the finish. Try after 2016 for optimum enjoyment.',
       'Mendoza Vineyards 2012 Gran Reserva by Richard Bonvin Malbec (Mendoza) Charred, smoky, herbal aromas of blackberry trend in the direction of exotic before showing barrel-driven scents of vanilla and clove. This exhibits plenty of depth and sponginess along with toasty, herbal flavors of plum and blackberry. Pepper, burnt toast, clove and blueberry flavors carry the finish on this hefty Malbec; drink through 2021.',
       'Prime 2013 Chardonnay (Coombsville) Slightly sour and funky in earth, this is a restrained white aged in a majority of stainless steel, in addition to smaller percentages of oak and acacia. Crisp and minerally, marked by stone

In [0]:
preds = clf_pipe.predict(test)

In [98]:
preds

array([14,  9,  5, ...,  3, 20,  3])

In [0]:
preds = enc.inverse_transform(preds)

In [100]:
preds

array(['Pinot Noir', 'Malbec', 'Chardonnay', ..., 'Cabernet Sauvignon',
       'Rosé', 'Cabernet Sauvignon'], dtype=object)

In [0]:
data1['variety'] = pd.DataFrame(preds)

In [102]:
data1

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,@paulgwine,US,Boedecker Cellars 2011 Athena Pinot Noir (Will...,Nicely differentiated from the companion Stewa...,Athena,88,35.0,Oregon,Willamette Valley,Willamette Valley,Boedecker Cellars,Pinot Noir
1,@wineschach,Argentina,Mendoza Vineyards 2012 Gran Reserva by Richard...,"Charred, smoky, herbal aromas of blackberry tr...",Gran Reserva by Richard Bonvin,90,60.0,Mendoza Province,Mendoza,NaN,Mendoza Vineyards,Malbec
2,@vboone,US,Prime 2013 Chardonnay (Coombsville),"Slightly sour and funky in earth, this is a re...",NaN,87,38.0,California,Coombsville,Napa,Prime,Chardonnay
3,@wineschach,Argentina,Bodega Cuarto Dominio 2012 Chento Vineyard Sel...,"This concentrated, midnight-black Malbec deliv...",Chento Vineyard Selection,91,20.0,Mendoza Province,Mendoza,NaN,Bodega Cuarto Dominio,Malbec
4,@kerinokeefe,Italy,SassodiSole 2012 Brunello di Montalcino,"Earthy aromas suggesting grilled porcini, leat...",NaN,90,49.0,Tuscany,Brunello di Montalcino,NaN,SassodiSole,Sangiovese
...,...,...,...,...,...,...,...,...,...,...,...,...
20660,@gordone_cellars,US,Yorkville Cellars 2013 Rennie Vineyard Caberne...,"Clearly focused and fruit-driven, this wine ha...",Rennie Vineyard,91,34.0,California,Yorkville Highlands,North Coast,Yorkville Cellars,Cabernet Franc
20661,@laurbuzz,France,Château Ribaute 2015 Senhal d'Aric Rosé (Corbi...,Herbal tones of bay and rosemary are upfront o...,Senhal d'Aric,84,20.0,Languedoc-Roussillon,Corbières,NaN,Château Ribaute,Rosé
20662,@mattkettmann,US,Daou 2014 Reserve Cabernet Sauvignon (Paso Rob...,"Mocha cream, pencil shaving and dried herb aro...",Reserve,94,85.0,California,Paso Robles,Central Coast,Daou,Cabernet Sauvignon
20663,@wineschach,Spain,Peñascal 2011 Tempranillo Rosé (Vino de la Tie...,Loud citrus and berry aromas precede an overlo...,Tempranillo,80,9.0,Northern Spain,Vino de la Tierra de Castilla y León,NaN,Peñascal,Rosé


In [0]:
data1.to_csv('./output.csv')